# <center>Week 13 Class 1 <center>
<div align="right"> April 11 2023 </div>

<div class="alert alert-block alert-warning">


# Today's aims:
* **_What is your final project?_**    
* **_More on 1D ODEs, coupling excitable and oscilating elements_**
* **_Demonstration of the Briggs–Rauscher reaction (modification of the Belousov–Zhabotinsky reaction)_**
* (https://en.wikipedia.org/wiki/Briggs%E2%80%93Rauscher_reaction)
* **_dynamics of the FHN model in 1D. Propagation of waves (in Python and JS)_**
* **_wave dynamics of excitable systems_**
* **_Implicit integration in 1D (The Thomas Algorithm)_**    
 

## Periodic Boundary Conditions
$$
    u(x=0)=u(x=l)
$$
<img src="fig3.png"  width="300">

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import *


In [ ]:
# Since T(0)= T(num)  
# How do we handle the extra values on the laplacian at 0 and num 
# laplacian = u(i+1)-2*u(i)+u(i-1)
num=5
x=np.linspace(0,1,num,endpoint=True); 
T=np.zeros(x.shape) ;
T=x
for i in range(0,num):
    print(i,T[i])
  
print()    
print(T[-1])


laplacian for i=0  $\; (u[-2]-2u[0]+u[1])$

laplacian for i=num $ \; (u[1]-2u[-1]+u[-2])$

In [ ]:
# x is set between 0,1 by deviding the distance to 50 segments
num=50
x=np.linspace(0,1,num,endpoint=True); 
# Initial condition of u is given as zero everywhere in the domain.
# set all u values to zero
T=np.zeros(x.shape) ; 
T=x*(1-x)  # Vectorized way to make quadratic distribution
Tn=np.zeros(x.shape) ; 
D=1.
T0=.5;
Tl=2;
dt=.00015 ;
dx=x[2]-x[1];
Dlap=D*dt/(dx*dx)

noTimeSteps =500; # number of time steps
noCurves=20 ;      # number of curves to draw for the duration of the solution
every=floor(noTimeSteps/noCurves); # how many time-steps between curves
Th=np.zeros((T.shape[0],noCurves));
Tmid=np.zeros(noCurves);
k=0;

for j in np.arange(noTimeSteps):
    if (j%every == 0): # keep some history of u for plotting!
        Th[:,k]=T;  #update using vectorization instead of for loop
        Tmid[k]=T[25];
        k=k+1;

    for i in range(1,num-1):
        Tn[i]=T[i]+Dlap*(T[i-1]-2.*T[i]+T[i+1]) ;
        
    Tn[0]= T[i]+Dlap*(T[-2]-2.*T[0]+T[1]); 
    Tn[-1]=T[-1]+Dlap*(T[1]-2.*T[-1]+T[-2]);
    for i in range(0,num):
        T[i]=Tn[i] ;
       
    
print(Dlap);
plt.figure(figsize=(6,2))

plt.plot(x,Th);
plt.xlabel('Length')
plt.ylabel('Temperature')
plt.show();


In [ ]:
# How fast it takes to burn yourself with the stove?
<img src="fig4.png">
From skin data we can estimate that $D\approx 290 cm^2/s$.

Assuming that the skin width is about 500 microns and that you touch a pan that was in the overn at $200^0C \;(\approx 400^0F)$. 

The average reaction time of a person is 215 miliseconds, would it possible to move your hand fast enough to avoid being burnt?

- Discretize the tissue in 100 segments and estimate what $dt$ to use.
- How long would it take for the inner tissue of your finger to reach about $180^0C$?  

 ### Try this at home and see if you can tell me next class the value.
 And what is the average reaction time of a human?

### What values should we use for dx and dt?

In [ ]:
#D dt/dx^2 < .5
D=290 #cm^2/s
dx=.05/100 #500 microns (in units of cm) devided by 100 units
dt_max=dx*dx/(2*D)
print('dt no maximum than',dt,'seconds')

In [ ]:
# x is set between 0,1 by deviding the distance to 50 segments
num=100
x=np.linspace(0,1,num,endpoint=True); 
# Initial condition of u is given as zero everywhere in the domain.
# set all u values to zero
T=np.zeros(x.shape) ; 
Tn=np.zeros(x.shape) ; 
D=290;  #cm^2/s
Tl=200; # 200 C
dt=.0000000004; 
dx=.05/100;  # consider at least 500 microns thickness of the skin. 
Dlap=D*dt/(dx*dx)
print(Dlap)
noTimeSteps = 25000 #89000; # number of time steps
noCurves=20 ;      # number of curves to draw for the duration of the solution
every=floor(noTimeSteps/noCurves); # how many time-steps between curves
Th=np.zeros((T.shape[0],noCurves));
Tend=np.zeros(noCurves);
time=np.zeros(noCurves);
k=0;

for j in np.arange(noTimeSteps):
    if (j%every == 0): # keep some history of u for plotting!
        Th[:,k]=T;  #update using vectorization instead of for loop
        Tend[k]=T[1];
        time[k]=j*dt;
        k=k+1;

    for i in range(1,num-1):
        Tn[i]=T[i]+Dlap*(T[i-1]-2.*T[i]+T[i+1]) ;
        
    for i in range(1,num-1):
        T[i]=Tn[i] ;
        T[0]= T[1];
        T[num-1]=Tl;

plt.figure(figsize=(6,2))
print(noTimeSteps*dt);
print(Tmid[noCurves-1]);
plt.plot(x,Th);
plt.xlabel('Length')
plt.ylabel('Temperature')
plt.show();
plt.figure(figsize=(6,2))
plt.plot(time,Tend)
plt.xlabel('Length')
plt.ylabel('curve number')
plt.show();


# Propagation of waves

Reaction diffusion models such as predator pray when coupled in space via diffusion can produce propagating waves and Turing Patterns.

## Recall the neuron model from Fitzhugh-Nagumo

#### FHN model a two variable neuronal model, and one of the  simple generic excitable sytem models.

$$\frac{du}{dt}= u(1-u)(u-a)-v$$


$$\frac{dv}{dt}=\epsilon\;(bu-v+\delta)$$
a=0.1,b=0.3, $\delta$=0.0, $\epsilon$=0.01
### From Week 8 Class 2:
<img src="fig5.png">

## 1D FitzHugh-Nagumo Model
#### We add the diffusion to the voltage variable
$$
    \frac{\partial u}{\partial t} = D \frac{\partial^2 u}{\partial t} + u(1.-u)(u-a)-v  $$
$$    \frac{\partial v}{\partial t} = \epsilon (bu-v+\delta)
$$
#### How do we integrate these two equations numerically?
a=0.1,b=0.3, $\delta$=0.0, $\epsilon$=0.01

- This model only diffusses in voltage, why?
- what would happen for models, like predator pray with 2 and 3 variables?

Based on the single cell code below, make a 1D code for the neuron model and investigate the dynamics.
- how will it differe from the termal problem?
- what happens if you use the oscillation regime? 
- what happens when you stimulate one part of the tissue

In [ ]:
## single cell code:
# Use this single cell FHN model from week 8 class 2 
#to make a 1D cable of cells  
eps=.05;
a=0.1;
b=0.3;
delta=0 #-0.045
time=400;
dt=0.01;
ntime=int(time/dt);
t= np.zeros(ntime);
u= np.zeros(ntime);
v= np.zeros(ntime);

u[0]=0.5;
v[0]=0;

for i in range(1,ntime):
    if(i % 2700 == 1):
        u[i-1]=.8
        
    t[i]=i*dt;
    u[i]=u[i-1]+dt*(u[i-1]*(1-u[i-1])*(u[i-1]-a)-v[i-1]);
    v[i]=v[i-1]+dt*(eps*(b*u[i-1]-v[i-1]+delta));

plt.figure(figsize=(6,2))
plt.plot(t,u,label='u voltage')
plt.plot(t,v,label='ion channel')

plt.ylim(-0.3, 1)
plt.xlim(0., time)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()  

Note that in the previous code u[i] and v[i], i is the counter for time. <br>
Now we will use i for space and evelove all of them in time, using a help variable ut[i]<br>
#### Rhus our integration will be:
$$u^{t+1}[i]=u^t[i]+\frac{Ddt}{dx^2}(u^t[i-1]+u^t[i+1]-2u^t[i])+dtf(u^t[i],v^t[i])$$

In [ ]:
#values to use:
1*.01/(.2*.2)


In [ ]:
#FHN model
eps=.025;
a=0.1;
b=0.3;
delta=0 #-0.045
time=140;
dt=0.01;
ntime=int(time/dt);

D=1
dx=.25
Dlap=D*dt/(dx*dx)
num=200
x=np.linspace(0,1,num,endpoint=True); 
u= np.zeros(num);
ut= np.zeros(num);
v= np.zeros(num);
noCurves=30 ;      # number of curves to draw for the duration of the solution
every=floor(ntime/noCurves); # how many time-steps between curves
up=np.zeros((u.shape[0],noCurves));
Dlap=D*dt/(dx*dx)

for i in range(0,10): ## excite the first 10 cells.
     u[i]=.9
       
    
m=0
for k in range(1,ntime):
    if (k%every == 0): # keep some history of u for plotting!
        up[:,m]=u;  #update using vectorization instead of for loop
        m=m+1;
        print(m,'out of',noCurves)
    #if(i % 3000 == 1):
    #    u[0]=.8
        
    for i in range(1,num-1):
        ut[i]=
        v[i]=v[i]+dt*(eps*(b*u[i]-v[i]+delta));     
        
    for i in range(1,num-1):
        u[i]=ut[i] ;
        u[0]= u[1];
        u[num-1]=u[num-2];        
            
plt.figure(figsize=(6,2))   
plt.plot(x,up);
plt.xlabel('Length')
plt.ylabel('voltage')
plt.show();


### What are time space plots?

In [ ]:
plt.figure(figsize=(6,4))  
deltaa=0
for k in range(1,noCurves):
    plt.plot(x,deltaa+up[:,k]);
    deltaa=deltaa+.1
#plt.plot(x,.1+up[:,1]);
plt.xlabel('Length')
plt.ylabel('time')
plt.show();

In [ ]:
Make an interactive "slider" to visualize what is happening.

Use the interactive plot section from Week 8 class 1
<img src="fig6.png">


In [ ]:
import numpy as np
from ipywidgets import interactive
import matplotlib.pyplot as plt
  
def f(time=0):  # make time=0 starts the slider at value 0
    plt.figure(figsize=(9,3)) 
    plt.plot(x,up[:,time]);
    plt.xlabel('Length')
    plt.ylabel('voltage')
    plt.ylim(-.5,1.2)
    plt.xlim(0,1)
    plt.show();

interactive_plot = interactive(f, time=(0, noCurves-1, 1))
output = interactive_plot.children[-1]
output.layout.height = '300px'
interactive_plot

### What happens when two waves collide?

### Now, convert your code to be used with periodic boundary conditions


In [ ]:
#FHN model
eps=.025;
a=0.1;
b=0.3;
delta=0 #-0.045
time=200;
dt=0.01;
ntime=int(time/dt);

D=1
dx=.25
Dlap=D*dt/(dx*dx)
num=400
x=np.linspace(0,1,num,endpoint=True); 
u= np.zeros(num);
ut= np.zeros(num);
v= np.zeros(num);
noCurves=30 ;      # number of curves to draw for the duration of the solution
every=floor(ntime/noCurves); # how many time-steps between curves
up=np.zeros((u.shape[0],noCurves));
Dlap=D*dt/(dx*dx)

for i in range(90,110):
     u[i]=.8
     #v[i-10]=.2   
    
m=0
for k in range(1,ntime):
    if (k%every == 0): # keep some history of u for plotting!
        up[:,m]=u;  #update using vectorization instead of for loop
        m=m+1;
    #if(i % 3000 == 1):
    #    u[0]=.8
        
    for i in range(1,num-1):
        ut[i]=u[i]+Dlap*(u[i-1]-2*u[i]+u[i+1])+dt*(u[i]*(1-u[i])*(u[i]-a)-v[i]);
        v[i]=v[i]+dt*(eps*(b*u[i]-v[i]+delta));
       
    ut[0]=u[i]+Dlap*(u[-2]-2*u[0]+u[1])+dt*(u[i]*(1-u[i])*(u[i]-a)-v[i]);
    ut[-1]=u[-1]+Dlap*(u[1]-2*u[-1]+u[-2])+dt*(u[i]*(1-u[i])*(u[i]-a)-v[i]);
    v[0]=v[0]+dt*(eps*(b*u[0]-v[0]+delta));
    v[-1]=v[-1]+dt*(eps*(b*u[-1]-v[i]+delta));

    for i in range(0,num):
        u[i]=ut[i] ;
        
plt.figure(figsize=(9,2))   
plt.plot(x,up);
plt.xlabel('Length')
plt.ylabel('voltage')
plt.show();


Make an interactive plot

In [ ]:
import numpy as np
from ipywidgets import interactive
import matplotlib.pyplot as plt

def f(time=0):
    plt.figure(figsize=(9,3))   
    plt.plot(x,up[:,time]);
    plt.xlabel('Length')
    plt.ylabel('voltage')
    plt.ylim(-.5,1.2)
    plt.xlim(0,1)
    plt.show();

interactive_plot = interactive(f, time=(0, noCurves-2, 1))
output = interactive_plot.children[-1]
output.layout.height = '250px'
interactive_plot

Refratory periods are very important in the dynamics of excitable system.

In [ ]:
#FHN model
eps=.025;
a=0.1;
b=0.3;
delta=0 #-0.045
time=300;
dt=0.01;
ntime=int(time/dt);

D=1
dx=.25
Dlap=D*dt/(dx*dx)
num=400
x=np.linspace(0,1,num,endpoint=True); 
u= np.zeros(num);
ut= np.zeros(num);
v= np.zeros(num);
noCurves=40 ;      # number of curves to draw for the duration of the solution
every=floor(ntime/noCurves); # how many time-steps between curves
up=np.zeros((u.shape[0],noCurves));
Dlap=D*dt/(dx*dx)

for i in range(90,110):
     u[i]=.8
     v[i-15]=.2   
    
m=0
for k in range(1,ntime):
    if (k%every == 0): # keep some history of u for plotting!
        up[:,m]=u;  #update using vectorization instead of for loop
        m=m+1;
    #if(i % 3000 == 1):
    #    u[0]=.8
        
    for i in range(1,num-1):
        ut[i]=u[i]+Dlap*(u[i-1]-2*u[i]+u[i+1])+dt*(u[i]*(1-u[i])*(u[i]-a)-v[i]);
        v[i]=v[i]+dt*(eps*(b*u[i]-v[i]+delta));
       
    ut[0]=u[i]+Dlap*(u[-2]-2*u[0]+u[1])+dt*(u[i]*(1-u[i])*(u[i]-a)-v[i]);
    ut[-1]=u[-1]+Dlap*(u[1]-2*u[-1]+u[-2])+dt*(u[i]*(1-u[i])*(u[i]-a)-v[i]);
    v[0]=v[0]+dt*(eps*(b*u[0]-v[0]+delta));
    v[-1]=v[-1]+dt*(eps*(b*u[-1]-v[i]+delta));

    for i in range(0,num):
        u[i]=ut[i] ;
        
plt.figure(figsize=(9,3))   
plt.plot(x,up);
plt.xlabel('Length')
plt.ylabel('voltage')
plt.show();


In [ ]:
import numpy as np
from ipywidgets import interactive
import matplotlib.pyplot as plt

def f(time=0):
    plt.figure(figsize=(9,3))   
    plt.plot(x,up[:,time]);
    plt.xlabel('Length')
    plt.ylabel('voltage')
    plt.ylim(-.5,1.2)
    plt.xlim(0,1)
    plt.show();

interactive_plot = interactive(f, time=(0, noCurves-2, 1))
output = interactive_plot.children[-1]
output.layout.height = '450px'
interactive_plot

### Implict integration in 1D for parabolic equations and reaction diffusion equations
Remember that Euler integration is: $$u^{t+1}[i]=u^t[i]+\frac{Ddt}{dx^2}(u^t[i+1]-2u^t[i]+u^t[i-1]+dtf(u^t[i],v^t[i])$$
implictily it will be to use the new time, at least for the diffusion part:
$$u^{\bf{t+1}}[i]=u^t[i]+\frac{Ddt}{dx^2}(u^{\bf{t+1}}[i+1]-2u^{\bf{t+1}}[i]+u^{\bf{t+1}}[i-1])+dtf(u^t[i],v^t[i])$$

therefore we have that:
   $$ (Ddt/dx^2)u^{\bf{t+1}}_{i+1}+(1-2Ddt/dx^2)u^{\bf{t+1}}_i+(Ddt/dx^2)u^{\bf{t+1}}_{i-1}=u^t_i + dt f(u^t_i , v^t_i ) $$ 
   
 this generates a 3D matrix that needs to be inverted to solve for $u^{t+1}_i$
 <img src="matrix.png"  width="300">
 Note that the top and bottom entry has only 2 components, because of the boundary conditions considered here, (zero flux).
 https://en.wikipedia.org/wiki/Tridiagonal_matrix_algorithm

in here:  $c_i$=$a_i$=$Ddt/dx^2$ except for $c_1=2Ddt/dx^2$ and $a_k=2Ddt/dx^2$<br>
$b_i=(1-2D dt/dx^2)$ and $d_i=u^t_i+dt f(u^t_i,v^t_i)$

Try this at home and if not we will code it next Tuesday

For Thursday, I have to go to a meeting at Emory. but this is good time for you durning class time to take the following online class I did with one of my postdoc to learn WebGL (JavaScript with interaction with the GPU) for ultrafast simulations of complex physical problems.
<br>
(https://github.com/kaboudian/WebGLTutorials)

(https://www.sciencedirect.com/science/article/pii/S0960077919300037)